Creare un'applicazione in Python che utilizza MySQL per gestire un sistema di libreria, includendo la gestione del prestito dei libri.

Setup del progetto:

Installare le librerie necessarie (mysql.connector per interagire con Mysql).
Creare un database chiamato libreria e una tabella chiamata libri opz(utenti).

Definizione del libro: Ogni libro deve avere i seguenti attributi:

titolo (string)
autore (string)
anno_pubblicazione (int)
genere (string)
disponibile (boolean)
prestiti (lista di oggetti che contengono informazioni sul prestito, come nome del prestatario e data del prestito)

Funzionalità dell'applicazione:

Aggiungi libro: Implementare una funzione che consente di aggiungere un nuovo libro alla collezione.
Visualizza libri: Implementare una funzione che mostra tutti i libri presenti nella collezione, evidenziando se sono disponibili o meno.
Modifica libro: Implementare una funzione che consente di modificare le informazioni di un libro esistente (identificato da titolo o autore).
Elimina libro: Implementare una funzione per rimuovere un libro dalla collezione.
Prestito libro: Implementare una funzione che consente di prestare un libro a un utente. Dovrà aggiornare lo stato del libro e registrare le informazioni del prestito.
Restituzione libro: Implementare una funzione che consente di restituire un libro, aggiornando lo stato del libro e rimuovendo il prestito dalla lista.

Interfaccia utente:

Creare un semplice menu a console che permetta all'utente di scegliere quale operazione eseguire (aggiungere, visualizzare, modificare, eliminare, prestare o restituire un libro).
Gestire le eccezioni per garantire che l'input dell'utente sia valido.

In [31]:
from datetime import datetime
import mysql.connector as mc

db = mc.connect(
    host='localhost',
    user='root',
    password='root',
    database='eserciziolibreria'
)

cursor = db.cursor() # Oggetto capace di comunicare con il DB

sql = 'CREATE TABLE IF NOT EXISTS libri (\
                id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,\
                titolo VARCHAR(50) NOT NULL,\
                autore VARCHAR(50) NOT NULL,\
                anno_pubblicazione INT NOT NULL,\
                genere VARCHAR(50) NOT NULL,\
                disponibile BOOLEAN default true)'
cursor.execute(sql)

sql = 'CREATE TABLE IF NOT EXISTS utenti (\
                id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,\
                nome VARCHAR(50) NOT NULL,\
                cognome VARCHAR(50) NOT NULL,\
                cf VARCHAR(50) NOT NULL,\
                telefono VARCHAR(50) NOT NULL)'
cursor.execute(sql)

sql = 'CREATE TABLE IF NOT EXISTS prestiti (\
                id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,\
                id_utente INT NOT NULL,\
                id_libro INT NOT NULL,\
                data TIMESTAMP NOT NULL)'
cursor.execute(sql)


def aggiungi_libro():
    titolo = input(f"Inserisci titolo")
    autore = input(f"Inserisci autore")
    genere = input(f"Inserisci genere")
    anno_pubblicazione = int(input(f"Inserisci anno di pubblicazione"))
    
    sql = 'INSERT INTO libri\
                    (titolo, autore, anno_pubblicazione, genere)\
                    VALUES (%s, %s, %s, %s)'
    values = (titolo, autore, anno_pubblicazione, genere)
    cursor.execute(sql, values)
    db.commit()
    if cursor._last_insert_id:
        print(f"Libro inserito nel DB")
        
#aggiungi_libro()

def visualizza_libri():
    sql = 'SELECT * FROM libri'
    cursor.execute(sql)
    listaLibri = cursor.fetchall()
    
    for libro in listaLibri:
        stato = 'Disponibile' if libro[5] else 'Non Disponibile'
        print(f"Libro {libro[1]}({libro[3]}) Autore: {libro[2]} \
            Genere: {libro[4]} Stato: {stato}")

# visualizza_libri()

def visualizza_libro(titolo) :
    sql = f"SELECT * FROM libri WHERE titolo = '{titolo}'"
    cursor.execute(sql)
    libro = cursor.fetchone()
    return libro
    
# visualizza_libro("Il passato è un morto senza cadavere")

def modifica_libro():
    titolo = input("Inserisci il titolo del libro che vuoi modificare")
    l = visualizza_libro(titolo)
    if l:
        titolo = input(f"Inserisci nuovo titolo ({l[1]})") or l[1]
        autore = input(f"Inserisci nuovo autore ({l[2]})") or l[2]
        anno_pubblicazione = int(input(f"Inserisci nuovo anno di pubblicazione ({l[3]})")) or l[3]
        genere = input(f"Inserisci nuovo genere ({l[4]})") or l[4]
        
        sql = 'UPDATE libri SET \
                    titolo = %s, autore = %s, \
                    anno_pubblicazione = %s, genere = %s \
                WHERE id = %s'
        values = (titolo, autore, anno_pubblicazione, genere, l[0])
        cursor.execute(sql, values)
        db.commit()
        
        print(f"Libro {l[1]} modificato nel DB")
    else:
        print(f"Libro {titolo} non trovato!!")
        
# modifica_libro()

def elimina_libro():
    titolo = input("Inserisci il titolo del libro che vuoi eliminare")
    sql = 'DELETE FROM libri WHERE titolo = %s'
    values = (titolo,)
    cursor.execute(sql, values)
    db.commit()
    
    # print(cursor.rowcount, "record(s) deleted")
    if cursor.rowcount > 0:
        print(f"Libro {titolo} eliminato nel DB")
    else:
        print(f"Libro {titolo} non trovato!!")

# elimina_libro()

def prestito_libro():
    titolo = input("Inserisci il titolo del libro che vuoi prendere in prestito")
    l = visualizza_libro(titolo)
    if l and l[5]:
        
        data_prestito = datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
        
        # Se vogliamo utilizzare le reference
        cf = input(f"Inserisci il Codice Fiscale")
        sql = f"SELECT * FROM utenti WHERE cf = '{cf}'"
        cursor.execute(sql)
        u = cursor.fetchone()
        
        if not u:
            utente = {
                "nome": input(f"Inserisci il nome"),
                "cognome": input(f"Inserisci il cognome"),
                "cf": cf,
                "telefono": input(f"Inserisci il numero di telefono")
            }
            
            sql = 'INSERT INTO utenti\
                    (nome, cognome, cf, telefono)\
                    VALUES (%s, %s, %s, %s)'
            values = (utente['nome'], utente['cognome'], utente['cf'], utente['telefono'])
            cursor.execute(sql, values)
            db.commit()
      
            print(cursor._last_insert_id, "record(s) created")
        else:
            print(u['nome'], u['cognome'])
            
# prestito_libro()